In [1]:
import pandas as pd
from pathlib import Path

PROVIDER = 'Biomedicum Imaging Unit'
INVOICE_DIR = '/work/data/OpenIRIS/BIU-20200916'
INVOICE_NAME = 'Invoice18_fixed.xlsx'

INVOICE_DIR = Path(INVOICE_DIR)
if not INVOICE_DIR.is_dir():
    raise ValueError('Please check INVOICE_DIR.')
INVOICE_FILE = INVOICE_DIR / INVOICE_NAME
if not INVOICE_FILE.exists():
    raise ValueError('Please check INVOICE_FILE.')

ERITTELYT = INVOICE_DIR / 'erittelyt'
ERITTELYT.mkdir(exist_ok=True)

df = pd.read_excel(INVOICE_FILE)

In [2]:
WBS = 'Cost center code'
WBSs = df[WBS].unique()

In [3]:
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment, Font
from openpyxl.styles.borders import Border, Side

wb = Workbook()

df.sort_values(by=['Group', WBS], inplace=True)

begin = df['Booking start'].min().strftime('%d.%m.%Y')
end = df['Booking end'].max().strftime('%d.%m.%Y')

cols = ['Booking start', 'Booking end', 'Resource', 'User name', 'Price', 'Qty', 'Charge', \
        'Price item', 'Request Title', 'Discount', 'Comments (charge)', 'Product comments', \
        ]

def prepare_sheet(ws, tmp):
    group = tmp['Group'][0]
    email = tmp['Group head(s) text'][0]
    billing_address = ''
    organization = tmp['Organization'][0]
    cost_center = tmp['Cost center code'][0]
    remit_code = tmp['Remit code'][0]
    price_type = tmp['Price type'][0]
    total_sum = tmp['Charge'].sum()
    
    ws.title = group + ' ' + str(wbs)
    
    ws['C1'] = PROVIDER
    ws['C3'] = 'Billing cycle:'
    ws['D3'] = str(begin) + '-' + str(end)
    ws['C4'] = 'Group/Payer:'
    ws['D4'] = group
    ws['C5'] = 'Email address:'
    ws['D5'] = email
    ws['C6'] = 'Billing address:'
    ws['D6'] = billing_address
    ws['C7'] = 'Organization:'
    ws['D7'] = organization
    ws['C8'] = 'Cost center:'
    ws['D8'] = cost_center
    ws['C9'] = 'Remit code:'
    ws['D9'] = remit_code
    ws['C10'] = 'Price type:'
    ws['D10'] = price_type
    ws['C11'] = 'Total sum:'
    ws['D11'] = total_sum
    ws['D11'].number_format = '#,##0.00€'
    ws['D12'] = ''
    ws['D13'] = ''
    
    tmp = tmp[cols]
    for r in dataframe_to_rows(tmp, index=False, header= True):
        ws.append(r)
        
    ws.cell(column=7,row=ws.max_row+1).value = total_sum
    
    ws.column_dimensions['A'].width = 18
    ws.column_dimensions['B'].width = 18
    ws.column_dimensions['C'].width = 25
    ws.column_dimensions['D'].width = 25
    ws.column_dimensions['E'].width = 5
    ws.column_dimensions['F'].width = 5
    ws.column_dimensions['G'].width = 10
    ws.column_dimensions['H'].width = 15
    ws.column_dimensions['I'].width = 50
    ws.column_dimensions['J'].width = 10
    ws.column_dimensions['K'].width = 40
    ws.column_dimensions['L'].width = 30
            
    # price column
    for cell in ws['E']:
        cell.number_format = '###0'

    # qty column
    for cell in ws['F']:
        cell.number_format = '###0.0'

    # charge column
    for cell in ws['G']:
        cell.number_format = '###0.00€'
        
    
    # change font
    mr = ws.max_row
    mc = ws.max_column
    for i in range(1,mr+1):
        for j in range(1,mc+1):
            ws.cell(row=i,column=j).font = Font(name='Calibri')

    # total sum
    ws['C11'].font = Font(name='Calibri', size='14', bold=True)
    ws['D11'].font = Font(name='Calibri', size='14', bold=True)

    # header row
    border = Border(top=Side(style='thin'), bottom=Side(style='thin'))
    for cell in ws['14']:
        cell.font = Font(name='Calibri', bold=True)
        cell.border = border
        cell.alignment = Alignment(horizontal='center')
        
    # date formats
    for col in [1,2]:
        for row in range(15,mr+1):
            ws.cell(row=row,column=col).number_format = 'YYYY-MM-DD HH:mm'
        

   
for wbs in WBSs:
    tmp = df[df[WBS] == wbs]
    tmp = tmp.reset_index()
    
    #print(wbs)
    if tmp['Group'].size == 0:
        print('no group found')
        continue
    
    # add new sheet to main workbook
    ws = wb.create_sheet()
    prepare_sheet(ws, tmp)

    # create separate workbook per wbs
    wb2 = Workbook()
    ws2 = wb2.active
    prepare_sheet(ws2, tmp)
    wb2.save(ERITTELYT / (ws2.title + '.xlsx'))
    
wb.save(INVOICE_DIR / (INVOICE_FILE.stem + '_split.xlsx'))
    
    

/home/hajaalin/anaconda3/envs/odf/lib/python3.8/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


no group found
